# Constrained minimization example

We consider the simple example from the <b>Gloptipoly</b> manual,

\begin{array}{ll}
\text{minimize} & -2 x_1 + x_2 - x_3\\
\text{subject to} & 24 - 20 x_1 + 9 x_2 - 13 x_3 + 4 x_1^2 - 4 x_1 x_2 + 4 x_1 x_3 + 2 x_2^2 - 2 x_2 x_3 + 2 x_3^2 \geq 0\\
& x_1 + x_2 + x_3 \leq 4, \quad 3 x_2 \leq 6\\
& 0 \leq x_1 \leq 2, \quad 0 \leq x_2, \quad 0\leq x_3\leq 3
\end{array}

In [2]:
include("..\\src\\Polyopt.jl")

Main.Polyopt

We define the variables

In [3]:
x1, x2, x3 = Polyopt.variables(["x1","x2","x3"]);

and the objective function

In [4]:
f = -2*x1 + x2 - x3;

and an array with all inequalities in the form $g_i(x) \geq 0$,

In [5]:
g = [24 - 20*x1 + 9*x2 - 13*x3 + 4*x1^2 - 4*x1*x2 + 4*x1*x3 + 2*x2^2 - 2*x2*x3 + 2*x3^2,
     4 - (x1 + x2 + x3),
     6 - (3*x2 + x3),
     x1, 2-x1, 
     x2, 
     x3, 3-x3]

8-element Array{Main.Polyopt.Poly{Int64},1}:
 24-13*x3+2*x3^2+9*x2-2*x2*x3+2*x2^2-20*x1+4*x1*x3-4*x1*x2+4*x1^2
 4-x3-x2-x1
 6-x3-3*x2
 x1
 2-x1
 x2
 x3
 3-x3

## Solving the second-order moment relaxation

This problem has multiple optimal minima, so we perturb the problem to locate one of the minimizers,

In [8]:
prob = Polyopt.momentprob(2, f + 1e-3*(x1+x2+x3), g);

We next solve the problem using MOSEK, turning off log-output:

In [9]:
X, Z, t, y, solsta = Polyopt.solve_mosek(prob, showlog=false);

In [10]:
t

-5.688381018039616

## Solving the third-order moment relaxation

In [13]:
prob = Polyopt.momentprob(3, f + 1e-3*(x1+x2+x3), g);

In [14]:
X, Z, t, y, solsta = Polyopt.solve_mosek(prob, showlog=false);

The lower bound decreases going from order two to order three,

In [15]:
t

-4.064821219921882

## Solving the fourth-order order moment relaxation

In [17]:
prob = Polyopt.momentprob(4, f + 1e-3*(x1+x2+x3), g);

We solve the fourth-order relaxation, this time showing the log-output from the solver,

In [18]:
X, Z, t, y, solsta = Polyopt.solve_mosek(prob, showlog=true);

Open file 'polyopt.task'
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 165             
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 9               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                  

Again the lower increases when we increase the relaxation order,

In [19]:
t

-3.997999988723318

At this point the hierarchy of relaxations has converged; higher order relaxations give the same bound. Let us inspect the recovered solution

In [20]:
[prob.basis y]

165×2 Array{Main.Polyopt.Poly{Float64},2}:
 1.0           1.0
 x3            6.21933e-6
 x3^2          1.86508e-5
 x3^3          5.59452e-5
 x3^4          0.000167824
 x3^5          0.000503452
 x3^6          0.00151031
 x3^7          0.00453085
 x3^8          14.0542
 x2            7.1917e-9
 x2*x3         1.31122e-8
 x2*x3^2       3.52898e-8
 x2*x3^3       1.01289e-7
 ⋮             
 x1^5*x2^2*x3  -0.560582
 x1^5*x2^3     1.36191
 x1^6          63.9999
 x1^6*x3       1.72981e-7
 x1^6*x3^2     3.18681
 x1^6*x2       1.26213e-7
 x1^6*x2*x3    -0.340108
 x1^6*x2^2     3.15167
 x1^7          128.0
 x1^7*x3       -2.07667
 x1^7*x2       2.01717
 x1^8          265.979

Let us verify that the recovered solution

In [21]:
X1, X2, X3 = Polyopt.vectorize([x1,x2,x3], 8)*y

3-element Array{Float64,1}:
 1.9999968898751308
 7.191695308455912e-9
 6.21932671434791e-6

 is optimal. It achieves the same objective value as the relaxation,

In [22]:
Polyopt.evalpoly(f, [X1; X2; X3])

-3.9999999918852804

and it is feasible,

In [23]:
[ Polyopt.evalpoly(gi, [X1; X2; X3]) for gi=g ]

8-element Array{Float64,1}:
 -1.8648903722606747e-5
  1.9999968836064594
  5.9999937590982
  1.9999968898751308
  3.110124869198927e-6
  7.191695308455912e-9
  6.21932671434791e-6
  2.9999937806732855